# Visualización Atómica
Por Benjamín Astete

---
Este archivo contiene el proceso completo de carga, procesamiento y visualización de los datos económicos de Chile (PIB y tasa de desempleo) utilizando Altair para nuestra investigación.

In [ ]:
# --- 1. Instalación de dependencias ---
!pip install altair vega_datasets pandas matplotlib cairosvg

In [110]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

# Assuming the uploaded file name is the key in the 'uploaded' dictionary
file_name = list(uploaded.keys())[0]

df = pd.read_csv(file_name, encoding='latin-1')
display(df.head())

Saving benjamin_database_limpia_01(Hoja1) (3).csv to benjamin_database_limpia_01(Hoja1) (3) (2).csv


,Período,Tasa de desempleo (Porcentaje),PIB variación respecto del trimestre anterior (%),IPC variación respecto del mes anterior (%),Unnamed: 4
0,dic-2024,8.1,12.37,-0.2,NaN
1,Nov-24,8.2,12.37,0.2,NaN
2,Oct-24,8.6,12.37,1.0,NaN
3,Sep-24,8.7,0.34,0.1,NaN
4,ago-2024,8.9,0.34,0.3,NaN


In [113]:
# --- 3. Procesamiento de datos ---
# Convertir el período a formato de fecha mensual (por ejemplo: 'dic-2024' → 2024-12-01)
df['Mes'] = pd.to_datetime(df['Período'], format='%b-%y', errors='coerce')
df = df.sort_values('Mes')

# --- Limpieza y conversión de PIB ---
df['PIB variación respecto del trimestre anterior (%) '] = (
    df['PIB variación respecto del trimestre anterior (%) ']
    .astype(str)
    .str.replace(',', '', regex=False)
)
df['PIB variación respecto del trimestre anterior (%) '] = pd.to_numeric(
    df['PIB variación respecto del trimestre anterior (%) '],
    errors='coerce'
)

# --- Limpieza y conversión de IPC ---
df['IPC variación respecto del mes anterior (%) '] = (
    df['IPC variación respecto del mes anterior (%) ']
    .astype(str)
    .str.replace(',', '', regex=False)
)
df['IPC variación respecto del mes anterior (%) '] = pd.to_numeric(
    df['IPC variación respecto del mes anterior (%) '],
    errors='coerce'
)

# --- Normalización de indicadores ---
df['PIB_normalizado'] = (
    (df['PIB variación respecto del trimestre anterior (%) '] - df['PIB variación respecto del trimestre anterior (%) '].min())
    / (df['PIB variación respecto del trimestre anterior (%) '].max() - df['PIB variación respecto del trimestre anterior (%) '].min())
)

df['Desempleo_normalizado'] = (
    (df['Tasa de desempleo (Porcentaje)'] - df['Tasa de desempleo (Porcentaje)'].min())
    / (df['Tasa de desempleo (Porcentaje)'].max() - df['Tasa de desempleo (Porcentaje)'].min())
)

df['IPC variación respecto del mes anterior (%)_normalizado'] = (
    (df['IPC variación respecto del mes anterior (%) '] - df['IPC variación respecto del mes anterior (%) '].min())
    / (df['IPC variación respecto del mes anterior (%) '].max() - df['IPC variación respecto del mes anterior (%) '].min())
)

display(df.head())



,Período,Tasa de desempleo (Porcentaje),PIB variación respecto del trimestre anterior (%),IPC variación respecto del mes anterior (%),Unnamed: 4,Año,PIB_normalizado,Desempleo_normalizado,IPC variación respecto del mes anterior (%)_normalizado,Mes
130,Feb-14,6.2,-2.53,0.5,NaN,2014-01-01,0.250170,0.041667,0.416667,2014-02-01
129,Mar-14,6.5,-2.53,0.8,NaN,2014-01-01,0.250170,0.083333,0.541667,2014-03-01
127,May-14,6.3,1.45,0.3,NaN,2014-01-01,0.385452,0.055556,0.333333,2014-05-01
126,Jun-14,6.5,1.45,0.1,NaN,2014-01-01,0.385452,0.083333,0.250000,2014-06-01
125,Jul-14,6.6,-3.19,0.2,NaN,2014-01-01,0.227736,0.097222,0.291667,2014-07-01


In [114]:
# --- 4. Visualización con Altair ---
import altair as alt

alt.data_transformers.disable_max_rows()

chart = alt.Chart(df).mark_line(point=True).encode(
    x=alt.X('Mes:T', title='Mes'),
    y=alt.Y('value:Q', title='Valor normalizado'),
    color=alt.Color('variable:N', title='Indicador'),
    tooltip=['Mes:T', 'variable:N', 'value:Q']
).transform_fold(
    [
        'PIB_normalizado',
        'Desempleo_normalizado',
        'IPC variación respecto del mes anterior (%)_normalizado'
    ],
    as_=['variable', 'value']
).properties(
    title='Evolución mensual del PIB, la Tasa de Desempleo y el IPC en Chile (2014–2024)',
    width=800,
    height=400
)

chart.interactive()


alt.Chart(...)

In [127]:
# --- 5. Exportación de visualización ---
import altair as alt
import cairosvg

# Exportar la visualización en HTML (formato interactivo)
chart.save('visualizacion_benjamin.html')

# Exportar primero como SVG (formato vectorial)
chart.save('visualizacion_benjamin.svg')

# Convertir el SVG a JPG usando cairosvg
cairosvg.svg2png(url='visualizacion_benjamin.svg', write_to='visualizacion_benjamin.jpg', dpi=300)

print("✅ Archivos exportados correctamente:")
print(" - visualizacion_benjamin.html (interactivo)")
print(" - visualizacion_benjamin.jpg (imagen estática)")


✅ Archivos exportados correctamente:
 - visualizacion_benjamin.html (interactivo)
 - visualizacion_benjamin.jpg (imagen estática)


In [126]:
!pip install vl-convert-python